# 데이터 전처리

In [2]:
from datetime import datetime
from tqdm import tqdm
import os

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_raw = pd.read_csv('../Lecture/파이썬_문법/Project_1205_09/data/csv/2019-Nov_1.csv',index_col=0)

In [3]:
df_raw.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,electronics,NaN,smartphone,NaN
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,appliances,NaN,sewing_machine,NaN
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,NaN,NaN,NaN,NaN
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,appliances,kitchen,washer,NaN
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,electronics,NaN,smartphone,NaN


## user_session == 1 값 제거
- user_session == 1인 경우, 유의미한 행동 패턴 분석이 어렵다 판단하여 1값 제거

In [4]:
# user_session 중복값 세기(v_c : value_count)

v_c = df_raw['user_session'].value_counts()
v_c

d99d91bf-40f8-4e29-9593-54b4a1826542    4128
fc749a4e-c432-4dae-a0a1-04de89f1e4ea    2466
b556f0c7-3a23-44f5-9f34-e713fefa9686    1963
d6433d7b-3846-456a-88de-748c3fac2675    1658
88206fc3-b5ea-4e3b-be68-67edfbf7009b    1373
                                        ... 
5f90efb4-8c4e-440f-aa88-bf1be39c19f7       1
e8e2f9ba-5cca-43b8-abe5-1b85279338b8       1
04294bee-917b-4152-9936-c2f36d67109f       1
f8ea0a0c-5c55-444b-9206-9c68ab18021d       1
90aca71c-ed8a-4670-866a-761ebacb732d       1
Name: user_session, Length: 13776050, dtype: int64

In [5]:
lst_v_c_idx = v_c.index.to_list()
lst_v_c = v_c.to_list()

In [6]:
dic = {}
for idx, value in zip(lst_v_c_idx, lst_v_c):
    dic[idx] = value

In [7]:
# try ~ except : Error idx 확인

lst_user_session_c = []
df_raw['user_session_c'] = 0
for idx in tqdm(range(len(df_raw))):
    try:
        df_raw['user_session_c'][idx] = dic[df_raw['user_session'][idx]]
    except:
        lst_user_session_c.append(idx)

100%|██████████| 67501979/67501979 [1:27:57<00:00, 12790.12it/s]


In [8]:
# user_session nan 값 또한 제거를 위해 1로 대입.
for idx in tqdm(lst_user_session_c):
    df_raw['user_session_c'][idx] = 0

100%|██████████| 10/10 [00:00<00:00, 2723.22it/s]


In [9]:
df = df_raw.query('user_session_c != 1')
df = df.query('user_session_c != 0')

## 상위 누적 구매액 10% 유저 추출

In [10]:
# 상품 구매 상위 10% 유저 구하기 위해 누적 금액 계산

df_pur_amount = df.query('event_type == "purchase"')
df_pur_amount = df_pur_amount.reset_index()
del df_pur_amount['index']
df_pur_amount.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4,user_session_c
0,2019-11-01 00:00:41 UTC,purchase,13200605,2053013557192163841,furniture.bedroom.bed,NaN,566.30,559368633,d6034fa2-41fb-4ac0-9051-55ea9fc9147a,furniture,bedroom,bed,NaN,3
1,2019-11-01 00:01:04 UTC,purchase,1005161,2053013555631882655,electronics.smartphone,xiaomi,211.92,513351129,e6b7ce9b-1938-4e20-976c-8b4163aea11d,electronics,NaN,smartphone,NaN,2
2,2019-11-01 00:04:51 UTC,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,128.42,562958505,0f039697-fedc-40fa-8830-39c1a024351d,electronics,NaN,smartphone,NaN,4
3,2019-11-01 00:05:34 UTC,purchase,26401669,2053013563651392361,NaN,lucente,109.66,541854711,c41c44d5-ef9b-41b9-9cd6-8d96dda6e927,NaN,NaN,NaN,NaN,3
4,2019-11-01 00:06:33 UTC,purchase,1801881,2053013554415534427,electronics.video.tv,samsung,488.80,557746614,4d76d6d3-fff5-4880-8327-e9e57b618e0e,electronics,video,tv,NaN,25


In [11]:
dic_acc_pri = {}
for idx in tqdm(range(len(df_pur_amount))):
    dic_acc_pri['{}'.format(df_pur_amount['user_id'][idx])] = 0
    
for idx in tqdm(range(len(df_pur_amount))):
    dic_acc_pri['{}'.format(df_pur_amount['user_id'][idx])] += df_pur_amount['price'][idx]

100%|██████████| 913677/913677 [00:04<00:00, 198764.48it/s]


In [12]:
# 개별 유저의 제품 누적 구매액

df_user = pd.DataFrame()

In [13]:
df_user['user_id'] = dic_acc_pri.keys()
df_user['acc_pur'] = dic_acc_pri.values()

In [14]:
df_user = df_user.sort_values(by =['acc_pur'],ascending=False)

In [15]:
n = (len(df_user)//10)
df_user_top10 = df_user.iloc[:int(n)]

In [16]:
df_user_top10.head()

,user_id,acc_purchase
230068,518514099,203986.07
2521,512386086,160092.24
5586,564068124,131929.20
969,515715331,117130.12
187449,521230795,111072.35


In [17]:
lst_user_top10 = df_user_top10['user_id'].to_list()
dic_user_top10 = {}
for idx in lst_user_top10:
    dic_user_top10[f'{idx}'] = 1

In [18]:
df['user_top10'] = 0
for idx in tqdm(range(len(df))):
    try:
        df['user_top10'][idx] = dic_user_top10['{}'.format(df['user_id'][idx])]
    except:
        continue

100%|██████████| 62501002/62501002 [07:41<00:00, 135342.25it/s]


In [19]:
df['user_top10'].value_counts()

0    59354089
1     3146913
Name: user_top10, dtype: int64

In [20]:
df_top10 = df.query('user_top10 == 1')
df_top10.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4,user_session_c,user_top10
59,2019-11-01 00:00:19 UTC,view,26100002,2053013553887052089,NaN,sulu,51.48,520956879,88e782d0-cb8f-4b25-b09c-6dd9bce06832,NaN,NaN,NaN,NaN,3,1
259,2019-11-01 00:01:46 UTC,view,39700019,2084962304016777941,apparel.costume,NaN,136.43,516426931,b36a7084-888d-4e37-bad4-cec9cfb1fbce,apparel,NaN,costume,NaN,4,1
290,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,electronics,NaN,smartphone,NaN,24,1
384,2019-11-01 00:02:36 UTC,view,5100816,2053013553375346967,NaN,xiaomi,29.58,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,NaN,NaN,NaN,NaN,24,1
434,2019-11-01 00:02:56 UTC,view,1005135,2053013555631882655,electronics.smartphone,apple,1665.36,562210838,a58d94c8-c0d4-4f24-bf3c-04c4e69ea153,electronics,NaN,smartphone,NaN,13,1


In [21]:
df_top10.describe()

,product_id,category_id,price,user_id,user_session_c,user_top10
count,3.146913e+06,3.146913e+06,3.146913e+06,3.146913e+06,3.146913e+06,3146913.0
mean,9.550945e+06,2.056449e+18,4.785720e+02,5.389170e+08,1.816475e+01,1.0
std,1.609361e+07,1.749123e+16,4.652784e+02,2.158344e+07,2.359861e+01,0.0
min,1.000978e+06,2.053014e+18,0.000000e+00,3.047076e+08,2.000000e+00,1.0
25%,1.005105e+06,2.053014e+18,1.277200e+02,5.164909e+08,5.000000e+00,1.0
50%,1.801967e+06,2.053014e+18,2.980700e+02,5.374373e+08,1.000000e+01,1.0
75%,1.271828e+07,2.053014e+18,7.454000e+02,5.600034e+08,2.100000e+01,1.0
max,1.000248e+08,2.186438e+18,2.574070e+03,5.780192e+08,3.190000e+02,1.0


In [22]:
df_top10['event_type'].value_counts()

view        2455529
cart         434693
purchase     256691
Name: event_type, dtype: int64

## feature 생성
### 카테고리 분류 나누기 : 대분류, 중분류, 소분류

In [23]:
# category_code 1: 대분류, 2 : 중분류, 3: 소분류, 4: 소소분류
# 카테고리 분류 결과 더 분류 가능하다 판단해 4까지 진행

df_top10['category_code_1'] = ''
df_top10['category_code_2'] = ''
df_top10['category_code_3'] = ''
df_top10['category_code_4'] = ''

In [24]:
for idx in tqdm(range(len(df_top10))):
    try:
        separate = df_top10['category_code'][idx].split('.')
        if len(separate) == 2:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_3'][idx] = separate[1]
        elif len(separate) == 3:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_2'][idx] = separate[1]
            df_top10['category_code_3'][idx] = separate[2]
        elif len(separate) == 4:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_2'][idx] = separate[1]
            df_top10['category_code_3'][idx] = separate[2]
            df_top10['category_code_4'][idx] = separate[3]
        else:
            print(idx,df_top10['category_code'])
    except:
        continue

100%|██████████| 3146913/3146913 [00:30<00:00, 104075.06it/s]


In [25]:
# category_code_1 NaNa 값 존재 : 3043814
df_top10['category_code_1'].value_counts()

                3043814
electronics       75669
appliances         8448
computers          8385
apparel            3712
furniture          3369
auto               1012
construction        898
kids                769
accessories         511
sport               253
medicine             59
country_yard         11
stationery            3
Name: category_code_1, dtype: int64

In [26]:
# category_code_2 NaNa 값 존재 : 3121077
df_top10['category_code_2'].value_counts()

               3121077
kitchen           6801
audio             5206
video             4412
living_room       1642
environment       1522
bedroom           1054
shoes             1022
accessories       1012
tools              899
peripherals        884
components         867
personal           262
camera             191
bathroom            28
fmcg                26
universal            5
furniture            3
Name: category_code_2, dtype: int64

In [27]:
# category_code_3 NaNa 값 존재 : 3043814
df_top10['category_code_3'].value_counts()

              3043814
smartphone      59909
notebook         5367
clocks           4699
tv               4392
               ...   
hammok              3
scarf               2
soldering           2
sock                1
step_ins            1
Name: category_code_3, Length: 117, dtype: int64

## Data NaN value check

In [28]:
def NaN_check(series):
    return series.isnull().sum()

In [29]:
NaN_check(df['event_time']),NaN_check(df['event_type'])

(0, 0)

In [30]:
NaN_check(df['product_id']),NaN_check(df['category_id'])

(0, 0)

In [31]:
NaN_check(df['price']),NaN_check(df['category_code']),NaN_check(df['brand']),

(0, 20268190, 8577955)

## NaN 값 처리

- 데이터에서 nan 값은 category_code, brand에 존재.
- 데이터에서 nan 값을 drop하는 방법도 고안했지만 user의 유의미한 행동 패턴을 분석하고자 가급적 nan값이 포함된 행을 살리는 방향으로 선택.
- 데이터 nan 값은 총 3가지 경우

||   경우의 수   |해결방안|
|:-------|:---------|----:|
|1|category_code만 nan 값 존재| 해당 행들의 brand 중 최다 빈도로 나온 category_code로 대입|
|2|brand만 nan 값 존재| 해당 행들의 category_code 중 최다 빈도로 나온 brand로 대입 |
|3|둘 다 nan 값 존재     | 둘 다 etc로 대입 |
|4|category_code만 nan 값이지만 해당 brand에 해당하는 모든 category_code의 값들이 nan값   |   해당 brand 값 그대로 대입 |

In [32]:
# TR : True & False

df_top10['TF_category_code'] = df_top10['category_code'].isnull()
df_top10['TF_brand'] = df_top10['brand'].isnull()

In [7]:
lst_nan = df_top10.query('TF_category_code == True').query('TF_brand == True').index.to_list()
lst_category_nan = df_top10.query('TF_category_code == True').query('TF_brand == False').index.to_list()
lst_brand_nan = df_top10.query('TF_category_code == False').query('TF_brand == True').index.to_list()

In [76]:
df_top10.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4,user_session_c,user_top10,weekday,separate_time,time
0,2019-11-01 00:00:19,view,26100002,2053013553887052089,sulu,sulu,51.48,520956879,88e782d0-cb8f-4b25-b09c-6dd9bce06832,NaN,NaN,NaN,NaN,3,1,Friday,0,0~3
1,2019-11-01 00:01:46,view,39700019,2084962304016777941,apparel.costume,nike,136.43,516426931,b36a7084-888d-4e37-bad4-cec9cfb1fbce,apparel,NaN,costume,NaN,4,1,Friday,0,0~3
2,2019-11-01 00:02:02,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,electronics,NaN,smartphone,NaN,24,1,Friday,0,0~3
3,2019-11-01 00:02:36,view,5100816,2053013553375346967,electronics.smartphone,xiaomi,29.58,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,NaN,NaN,NaN,NaN,24,1,Friday,0,0~3
4,2019-11-01 00:02:56,view,1005135,2053013555631882655,electronics.smartphone,apple,1665.36,562210838,a58d94c8-c0d4-4f24-bf3c-04c4e69ea153,electronics,NaN,smartphone,NaN,13,1,Friday,0,0~3


In [5]:
for idx in tqdm(lst_nan):
    df_top10['category_code'][idx] = 'etc'
    df_top10['brand'][idx] = 'etc'


100%|██████████| 211194/211194 [51:00<00:00, 69.00it/s] 


In [6]:

for idx in tqdm(lst_brand_nan):
    try:
        df_top10['brand'][idx] = df_top10.query('category_code == "{}"'.format(df_top10['category_code'][idx]))['brand'].value_counts().index[0]
    except:
        print(idx)

100%|██████████| 105588/105588 [1:37:12<00:00, 18.10it/s] 


In [8]:
df_top10['event_time'] = df_top10['event_time'].apply(lambda x:x[:19])

In [9]:
df_top10['weekday'] =0

In [10]:
df_top10['weekday'] = df_top10['event_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime("%A"))

In [11]:
df_top10['weekday']

0            Friday
1            Friday
2            Friday
3            Friday
4            Friday
             ...   
3146908    Thursday
3146909    Thursday
3146910    Thursday
3146911    Thursday
3146912    Thursday
Name: weekday, Length: 3146913, dtype: object

In [12]:
df_top10['separate_time'] = 0

In [13]:
df_top10['time'] = ''

In [14]:
for idx in tqdm(range(len(df_top10))):
    t = int(df_top10['event_time'][idx][11:13])
    if t < 3:
        df_top10['time'][idx] = '0~3'
    elif t >= 3 and t < 6:
        df_top10['time'][idx] = '3~6'
    elif t >= 6 and t < 9:
        df_top10['time'][idx] = '6~9'
    elif t >= 9 and t < 12:
        df_top10['time'][idx] = '9~12'
    elif t >= 12 and t < 15:
        df_top10['time'][idx] = '12~15'
    elif t >= 15 and t < 18:
        df_top10['time'][idx] = '15~18'
    elif t >= 18 and t < 21:
        df_top10['time'][idx] = '18~21'
    elif t >= 21 and t < 24:
        df_top10['time'][idx] = '21~24'

100%|██████████| 3146913/3146913 [04:21<00:00, 12015.07it/s]


## event_type 구별

- 1. view
- 2. cart
- 3. purchase

In [16]:
df_top10['user_id'] = df_top10['user_id'].apply(lambda x: str(x))

In [17]:
df_pur = df_top10.query('event_type == "purchase"')
df_view = df_top10.query('event_type == "view"')
df_cart = df_top10.query('event_type == "cart"')

In [18]:
df_pur.shape,df_cart.shape,df_view.shape

((256691, 22), (434693, 22), (2455529, 22))

In [19]:
df_pur = df_pur.reset_index()
df_view = df_view.reset_index()
df_cart = df_cart.reset_index()

del df_pur['index']
del df_view['index']
del df_cart['index']

In [20]:
lst_p_user_id = df_pur['user_id'].value_counts().index.to_list()
lst_v_user_id = df_view['user_id'].value_counts().index.to_list()
lst_c_user_id = df_cart['user_id'].value_counts().index.to_list()

In [21]:
dic_p_by_user_id = {}
dic_v_by_user_id = {}
dic_c_by_user_id = {}

In [22]:
for idx in lst_p_user_id:
    dic_p_by_user_id[f'{idx}'] = 0
for idx in lst_v_user_id:
    dic_v_by_user_id[f'{idx}'] = 0
for idx in lst_c_user_id:
    dic_c_by_user_id[f'{idx}'] = 0

In [23]:
for idx in tqdm(range(len(df_pur))):
    dic_p_by_user_id['{}'.format(df_pur['user_id'][idx])] += df_pur['price'][idx]
    
for idx in tqdm(range(len(df_view))):
    dic_v_by_user_id['{}'.format(df_view['user_id'][idx])] += df_view['price'][idx]
    
for idx in tqdm(range(len(df_cart))):
    dic_c_by_user_id['{}'.format(df_cart['user_id'][idx])] += df_cart['price'][idx]

100%|██████████| 434693/434693 [00:02<00:00, 209159.67it/s]


In [24]:
# df_user_total_price
df_user_total_price = pd.DataFrame()

In [25]:
df_user_total_price['user_id'] = dic_p_by_user_id.keys()
df_user_total_price['total_price'] = dic_p_by_user_id.values()

In [26]:
df_user_total_price = df_user_total_price.sort_values(by='total_price',ascending=False)

In [27]:
df_user_total_price.head()

,user_id,total_price
2,518514099,203567.53
1,512386086,138329.07
0,564068124,128594.36
65,515715331,115488.34
28,521230795,96336.84


In [28]:
# 상위 10% 상품 제품 누적 금액 유저 수 : 44039

df_user_total_price['user_id'].nunique()

42560

In [29]:
# 10그룹 나누기 위한 숫자

n = int(len(df_user_total_price)*0.1)

In [30]:
dic_group = {}
for ind in tqdm(range(n*9,len(df_user_total_price))):
    dic_group['{}'.format(df_user_total_price['user_id'][ind])] = 10

100%|██████████| 4256/4256 [00:00<00:00, 282250.89it/s]


In [31]:
df_pur['group'] = 0

In [32]:
df_pur['group'].value_counts()

0    256691
Name: group, dtype: int64

## per user_id dataframe

In [55]:
df_user_id = pd.DataFrame()

In [56]:
df_user_id['user_id'] = 0
df_user_id['view'] = 0
df_user_id['cart'] = 0
df_user_id['c_pur'] = 0 # purchase count
df_user_id['t_pur'] = 0 # purchase total amount
df_user_id['user_session_unique'] = 0 # per user session count
df_user_id['number_behavior'] = 0 # unique user_session count
df_user_id['v_brand'] = 0 # brand view
df_user_id['c_brand'] = 0 # brand cart 
df_user_id['p_brand'] = 0 # brand purchase 
df_user_id['weekday_max'] = 0
df_user_id['time_max'] = 0
df_user_id['view_price_avg'] = 0 
df_user_id['cart_price_avg'] = 0 
df_user_id['pur_price_avg'] = 0 
df_user_id['product_id_unique'] = 0 
df_user_id['pur_price_max'] = 0
df_user_id['pur_price_min'] = 0
df_user_id['c_category_code'] = 0
df_user_id['category_code_max'] = 0 
df_user_id['category_code_min'] = 0 
df_user_id['CV'] = 0  # view / cart
df_user_id['PC'] = 0  # purchase / cart
df_user_id['PV'] = 0  # purchase / view
df_user_id['VS'] = 0
df_user_id['CS'] = 0
df_user_id['PS'] = 0

In [57]:
df_user_id.shape

(0, 27)

In [77]:
lst_nan = {}
for idx, user in tqdm(enumerate(df_top10['user_id'].unique().tolist())):
    try:
        df_user_id.loc[idx] = [0 for _ in range(27)]
        df_user_id['user_id'][idx] = user
        a = df_top10.query(f'user_id == "{user}"')
        df_user_id['view'][idx] = len(a.query('event_type == "view"'))
        df_user_id['cart'][idx] = len(a.query('event_type == "cart"'))
        df_user_id['c_pur'][idx] = len(a.query('event_type == "purchase"'))
        df_user_id['t_pur'][idx] = sum(a.query('event_type == "purchase"')['price'])
        df_user_id['number_behavior'][idx] = len(a)
        df_user_id['user_session_unique'][idx] = a['user_session'].nunique()
        df_user_id['v_brand'][idx] = a.query('event_type == "view"')['brand'].nunique()
        df_user_id['c_brand'][idx] = a.query('event_type == "cart"')['brand'].nunique()
        df_user_id['p_brand'][idx] = a.query('event_type == "purchase"')['brand'].nunique()
        df_user_id['weekday_max'][idx] = a['weekday'].value_counts().index[0]
        df_user_id['time_max'][idx] = a['time'].value_counts().index[0]
        df_user_id['view_price_avg'][idx] =  a.query('event_type == "view"')['price'].mean()
        df_user_id['cart_price_avg'][idx] =  a.query('event_type == "cart"')['price'].mean()
        df_user_id['pur_price_avg'][idx] =  a.query('event_type == "purchase"')['price'].mean()
        df_user_id['product_id_unique'][idx] =  a['product_id'].nunique()
        df_user_id['pur_price_max'][idx] = max(a.query('event_type == "purchase"')['price'])
        df_user_id['pur_price_min'][idx] = min(a.query('event_type == "purchase"')['price'])
        df_user_id['CV'][idx] = (df_user_id['cart'][idx]/df_user_id['view'][idx])*100
        df_user_id['PV'][idx] = (df_user_id['c_pur'][idx]/df_user_id['view'][idx])*100
        df_user_id['VS'][idx] = len(a.query('event_type == "view"'))/a['user_session'].nunique()
        df_user_id['CS'][idx] = len(a.query('event_type == "cart"'))/a['user_session'].nunique()
        df_user_id['PS'][idx] = len(a.query('event_type == "purchase"'))/a['user_session'].nunique()
        try:
            df_user_id['category_code_max'][idx] =  a['category_code'].value_counts().index[0]
            df_user_id['category_code_min'][idx] =  a['category_code'].value_counts().index[-1]
            df_user_id['PC'][idx] = (df_user_id['c_purchase'][idx]/df_user_id['cart'][idx])*100
            df_user_id['c_category_code'][idx] = a['category_code_1'].nunique()
        except:
            df_user_id['category_code_max'][idx] =  'nan'
            df_user_id['category_code_min'][idx] =  'nan'
            df_user_id['PC'][idx] = 0
            df_user_id['c_category_code'][idx] = 0
    except:
        lst_nan['{}'.format(user)] = idx

43223it [37:08, 19.40it/s]


In [79]:
len(lst_nan)

663

In [75]:
df_top10.shape

(3146913, 18)